In [ ]:
'''
This Python file contains the object-oriented programming version of the following problem statement.

Given known information about the students of a University at the end of an academic year, namely personal information
such as unique indetifiers and their grades, this app is intended to help with the good record keeping for the admin
teams. While building this app, we are aiming for the following utilities:
    - list all students;
    - identify the passing and failing students;
    - given a certain grade threshold, identify the students below and above that grade;
    - show the students that have obtained a bursaries;
    - order the failing students based on the number of resits;
    - order the students based on the grade from a certain class;

Note: All exceptions will be correctly raised with a friendly user readable message.
'''

'\nThis Python file contains the object-oriented programming version of the following problem statement.\n\nGiven known information about the students of a University at the end of an academic year, namely personal information\nsuch as unique indetifiers and their grades, this app is intended to help with the good record keeping for the admin\nteams. While building this app, we are aiming for the following utilities:\n    - list all students;\n    - identify the passing and failing students;\n    - given a certain grade threshold, identify the students below and above that grade;\n    - show the students that have obtained a bursaries;\n    - order the failing students based on the number of resits;\n    - order the students based on the grade from a certain class;\n\nNote: The output of the programm will be easily writtable and readable from txt files. \nNote: All exceptions will be correctly raised with a friendly user readable message.\n'

In [1]:
# import packages
from datetime import date

In [2]:
# create a class that will manipulate the students in a particular year of study
class Year:
    # create instance variables with year, students, and courses
    def __init__(self, year):
        self.year = year    # integer value for the year
        self.students = []  # list with students
        self.courses = []   # list with courses taken
    

    # add each student or multiple students created with Student class to the list of students 
    def add_student(self, student):
        # add a Student object to students
        self.students.append(student)
    
    def add_students(self, students):
        # add multiple students
        for student in students:
            self.add_student(student)
    

    # display all students in the academic year in alphabetical order
    def showStudents(self, bursaries_only = False):
        if bursaries_only:
          students = [st for st in self.students if st.bursary]
        else:
          students = self.students

        for student in sorted(students, key=lambda x: x.full_name):
            print(str(student))
    

    # display all students that have to do resits and how many
    def showResits(self):
        students = [st for st in self.students if st.resits]

        for student in sorted(students, key=lambda x: x.resits):
            print(f'{str(student)} {student.resits}')
    

    # display all students that have to do resits and how many
    def bestPerformers(self, course):
        students = [st for st in self.students if st.resits]

        for student, final in sorted(course.final_marks.items(), key=lambda item: item[1], reverse = True):
            print(f'{str(student)} {final}')
    

    def add_course(self, course):
        # add a Course object to courses
        self.courses.append(course)
    
    def add_courses(self, courses):
        # add multiple courses
        for course in courses:
            self.add_course(course)
    
    
    def __repr__(self):
        return f'Year("{self.year}")'

In [3]:
class Student:
    # create instance variables with first name, last name, bursary, disability as private, and courses
    def __init__(self, first_name, last_name, bursary = False, disability = False):
        self.first_name = first_name
        self.last_name = last_name
        self.bursary = bursary
        self.__disability = disability  # private method to protect the information from being accessed
        self.courses = []   # list of courses for each student
    
    def join_course(self, course):
        # add course to student

        # avoid enrolling to same course twice
        if course not in self.courses:
            # add student to course
            course.add_grades(self)
            self.courses.append(course)
    

    @property
    def full_name(self):
        return self.last_name + ' ' + self.first_name
    
    @property
    def resits(self):
        return sum(course.passed[self] == 'FAIL' for course in self.courses)
    
    def __str__(self):
        return self.full_name
    
    def __repr__(self):
        return f'Student("{self.first_name}","{self.last_name}")'

In [4]:
class Course:
    # create instance variables with name, assignments, all_grades
    def __init__(self, name, assignments):
        self.name = name
        self.assignments = assignments
        self.all_grades = {}    # dictionary (student:grade)
    
    def add_grades(self,student):
        # collect marks for each assignment
        assignment_marks = {}

        for assignment in self.assignments:
            value = int(input(f'{assignment} Grade: '))
            assignment_marks[assignment] = value

        # add marks to student
        self.all_grades[student] = assignment_marks
    
    @property
    def final_marks(self):
        # get final (average) grade for each student 
        return {student: sum(assignment_marks.values())/len(assignment_marks)
                for student, assignment_marks in self.all_grades.items()}

    @property
    def passed(self):
        # categorise each student as pass or failed 
        return {student: ('PASS' if final >= 5 else 'FAIL')
                for student, final in self.final_marks.items()}
    
    # show students who passed and failed in alphabetical order
    def show_passed_and_failed(self):
        passed = []
        failed = []

        # check whether a student passed or failed and append them to 
          # their corresponding lists
        for student, result in self.passed.items():
            if result == 'PASS':
                passed.append(student)
            elif result == 'FAIL':
                failed.append(student)
            else:
                raise NotImplementedError
        
        # order students who passed
        for student in sorted(passed, key=lambda x: x.full_name):
            print(f'{str(student)} PASS')
        
        print()   # empty row

        # order students who failed
        for student in sorted(failed, key=lambda x: x.full_name):
            print(f'{str(student)} FAIL')


    def threshold(self, grade):
        above = []  # or equal
        below = []

        for student, final in self.final_marks.items():
            if final >= grade:
                above.append(student)
            else:
                below.append(student)
        
        for student in sorted(above, key=lambda x: x.full_name):
            print(f'{str(student)} >={grade}')
        
        print()   # empty row

        for student in sorted(below, key=lambda x: x.full_name):
            print(f'{str(student)} <{grade}')
    

    def __repr__(self):
        return f'Course("{self.name}")'

In [5]:
ana = Student('Ana', 'Popescu')
george = Student('George', 'Romanescu')
ion = Student('Ion', 'Axenie', bursary = False)
maria = Student('Maria', 'Ivrim', bursary = True)

maths = Course('Mathematics', assignments = ['Homework 1','Homework 2','Exam'])
compsci = Course('Computer Science', assignments = ['Project 1','Project 2','Project 3'])

this_year = Year(2022)
this_year.add_students([ana, george, ion, maria])
this_year.add_courses([maths, compsci])

In [6]:
this_year.students

[Student("Ana","Popescu"),
 Student("George","Romanescu"),
 Student("Ion","Axenie"),
 Student("Maria","Ivrim")]

In [7]:
this_year.courses

[Course("Mathematics"), Course("Computer Science")]

In [8]:
# list all students;
this_year.showStudents()

Axenie Ion
Ivrim Maria
Popescu Ana
Romanescu George


In [9]:
# show the students that have obtained a bursary;
this_year.showStudents(bursaries_only=True)

Ivrim Maria


In [10]:
ana.join_course(maths)    # grades: 3,4,5
george.join_course(maths) # grades: 7,8,9
ion.join_course(maths)    # grades: 6,7,7
maria.join_course(maths)  # grades: 10,10,9

Homework 1 Grade: 3
Homework 2 Grade: 4
Exam Grade: 5
Homework 1 Grade: 7
Homework 2 Grade: 8
Exam Grade: 9
Homework 1 Grade: 6
Homework 2 Grade: 7
Exam Grade: 7
Homework 1 Grade: 10
Homework 2 Grade: 10
Exam Grade: 9


In [11]:
# identify the passing and failing students;
maths.show_passed_and_failed()

Axenie Ion PASS
Ivrim Maria PASS
Romanescu George PASS

Popescu Ana FAIL


In [14]:
# given a certain grade threshold, identify the students below and above that grade;
maths.threshold(9)

Ivrim Maria >=9

Axenie Ion <9
Popescu Ana <9
Romanescu George <9


In [15]:
ana.join_course(compsci)      # grades: 4,4,6
george.join_course(compsci)   # grades: 6,9,8
ion.join_course(compsci)      # grades: 3,3,2
maria.join_course(compsci)    # grades: 9,9,10

Project 1 Grade: 4
Project 2 Grade: 4
Project 3 Grade: 6
Project 1 Grade: 6
Project 2 Grade: 9
Project 3 Grade: 8
Project 1 Grade: 3
Project 2 Grade: 3
Project 3 Grade: 2
Project 1 Grade: 9
Project 2 Grade: 9
Project 3 Grade: 10


In [18]:
compsci.show_passed_and_failed()

Ivrim Maria PASS
Romanescu George PASS

Axenie Ion FAIL
Popescu Ana FAIL


In [19]:
# - order the failing students based on the number of resits;
this_year.showResits()

Axenie Ion 1
Popescu Ana 2


In [20]:
# order the students based on the grade from a certain class;
this_year.bestPerformers(maths)

Ivrim Maria 9.666666666666666
Romanescu George 8.0
Axenie Ion 6.666666666666667
Popescu Ana 4.0


In [21]:
this_year.bestPerformers(compsci)

Ivrim Maria 9.333333333333334
Romanescu George 7.666666666666667
Popescu Ana 4.666666666666667
Axenie Ion 2.6666666666666665
